# GPT-OSS mit OpenWebUI in Google Colab

Dieses Notizbuch richtet das `gpt-oss:20b`-Modell mit Ollama ein und macht es über eine öffentliche OpenWebUI-Schnittstelle zugänglich.

**Anleitung:**
1. Stellen Sie sicher, dass Ihre Colab-Laufzeit auf eine GPU-Instanz eingestellt ist (`Laufzeit` > `Laufzeittyp ändern` > `T4 GPU`).
2. Führen Sie die Zellen nacheinander aus, indem Sie auf das "Play"-Symbol klicken oder `Shift + Enter` drücken.

In [ ]:
#@markdown Führen Sie diese Zelle aus, um alle erforderlichen Abhängigkeiten zu installieren.
#@markdown > ⏳ **Dies kann einige Minuten dauern.**
%%capture
print("⏳ Installation von Ollama, OpenWebUI und ngrok wird gestartet...")
!pip install open-webui pyngrok nest-asyncio
!curl -fsSL https://ollama.com/install.sh | sh
print("✅ Installation abgeschlossen!")

## 2. Server starten und auf die Web-UI zugreifen

In [ ]:
#@markdown Führen Sie diese Zelle aus, um Ollama und OpenWebUI zu starten.
#@markdown > ⏳ **Das Herunterladen des Modells kann eine Weile dauern.**
import os
import asyncio
import nest_asyncio
from pyngrok import ngrok

# Erlaubt die verschachtelte Verwendung von asyncio.run() in Umgebungen wie Colab
nest_asyncio.apply()

async def run_ollama():
    # Ollama im Hintergrund starten
    ollama_process = await asyncio.create_subprocess_shell(
        'ollama serve',
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    print("⏳ Ollama-Server wird gestartet...")
    # Kurz warten, bis der Server bereit ist
    await asyncio.sleep(5)

    # Modell herunterladen
    print("⏳ Lade das gpt-oss:20b Modell herunter. Dies kann einige Zeit dauern...")
    pull_process = await asyncio.create_subprocess_shell(
        'ollama pull gpt-oss:20b',
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    await pull_process.communicate()
    print("✅ Modell erfolgreich heruntergeladen!")

    # OpenWebUI starten
    print("⏳ OpenWebUI wird gestartet...")
    os.environ['OLLAMA_BASE_URL'] = 'http://127.0.0.1:11434'
    webui_process = await asyncio.create_subprocess_shell(
        'open-webui --host 0.0.0.0 --port 8080',
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )
    await asyncio.sleep(10) # Zeit geben zum Starten

    # Ngrok-Tunnel einrichten und öffentliche URL ausgeben
    try:
      public_url = ngrok.connect(8080)
      print(f"✅ OpenWebUI ist bereit! Öffnen Sie diesen Link in Ihrem Browser:\n{public_url}")
    except Exception as e:
      print(f"Fehler beim Starten von ngrok: {e}")
      print("Bitte fügen Sie Ihr ngrok-Authtoken in Colab hinzu (siehe Seitenleiste > Geheimnisse) und versuchen Sie es erneut.")
    
    # Prozesse am Leben halten
    await asyncio.gather(ollama_process.wait(), webui_process.wait())

# Führt die asynchrone Funktion aus
asyncio.run(run_ollama())